# Chapter 3 - Exercise 4
#### Author: *John Benedick Estrada*
---
**Exercise:** In the book, I claimed that Dijkstra's algorithm does not work unless it uses BFS.  Write a version of `shortest_path_dijkstra` that uses DFS and test it on a few examples to see what goes wrong.

In [1]:
import networkx as nx
from collections import deque
from typing import Hashable

##### Functions directly lifted from the book

In [2]:
def adjacent_edges(nodes, halfk):
    n = len(nodes)
    for i, u in enumerate(nodes):
        for j in range(i+1, i+halfk+1):
            v = nodes[j % n]
            yield u, v


def make_ring_lattice(n, k):
    G = nx.Graph()
    nodes = range(n)
    G.add_nodes_from(nodes)
    G.add_edges_from(adjacent_edges(nodes, k//2))
    return G


def shortest_path_dijkstra(G, source):
    """Finds shortest paths from `source` to all other nodes.
    
    G: graph
    source: node to start at
    
    returns: make from node to path length
    """
    dist = {source: 0}
    queue = deque([source])
    while queue:
        node = queue.popleft()
        new_dist = dist[node] + 1

        neighbors = set(G[node]).difference(dist)
        for n in neighbors:
            dist[n] = new_dist
        
        queue.extend(neighbors)
    return dist

##### My implementations of a DFS "shortest path" algorithm

In [31]:
# Stack based variant of a DFS "shortest path" algorithm.
def dfs_spa_stack(G: nx.Graph, source: Hashable):
    dist = {source: 0}
    stack = deque([source])
    while stack:
        # Get the topmost element on the stack which should be the most
        # recently visited one.
        node = stack[-1]

        for n in G[node]:
            # Check if `n` is already visited.
            # If not, update `dist` and visit that node.
            if n not in dist:
                new_dist = dist[node] + 1
                dist[n] = new_dist
                stack.append(n)
                break
        else:
            # If all the neighbors of the current node `node` have already been
            # visited, backtrack until a node with at least one unvisited
            # neighbor is reached.
            stack.pop()

    # A graph is fully visited if the stack is empty.
    return dist


# Recursion variant of a DFS "shortest path" algorithm.
def dfs_spa_recursive(G: nx.Graph, source: Hashable):
    dist = {source: 0}

    def visit(node: Hashable):
        for n in G[node]:
            # Check if `n` is already visited.
            # If not, update `dist` and visit that node.
            if n not in dist:
                new_dist = dist[node] + 1
                dist[n] = new_dist
                visit(n)

    visit(source)
    return dist

##### Testing the implementations

In [4]:
lattice = make_ring_lattice(100, 100)

In [32]:
spa_stack_result = dfs_spa_stack(lattice, 0)
spa_recursive_result = dfs_spa_recursive(lattice, 0)
assert spa_stack_result == spa_recursive_result
print("Success!")

del spa_stack_result, spa_recursive_result

Success!


##### Just a simple benchmark between the two implementations

In [34]:
print("Stack based implementation:     ", end="")
%timeit dfs_spa_stack(lattice, 0)
print("Recursion based implementation: ", end="")
%timeit dfs_spa_recursive(lattice, 0)

Stack based implementation:     578 µs ± 14.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
Recursion based implementation: 373 µs ± 7.02 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


##### Test the result of the DFS "shortest path" algorithm

In [7]:
def cmpr_two_spa_result(n, k):

    lattice = make_ring_lattice(n, k)
    dfs_spa_result = dfs_spa_recursive(lattice, 0)   # We use the recursive one since it is faster (in my machine at least)
    true_spa_result = shortest_path_dijkstra(lattice, 0)

    header = "Node : DFS SPA result : True SPA result"
    title = f"Lattice (n={n}, k={k})"
    print(f"{title}\n{'-'*len(header)}\n{header}\n{'-'*len(header)}")

    for node in lattice.nodes:
        print(f"{node} : {dfs_spa_result[node]} : {true_spa_result[node]}", 
              end="")
        if (dfs_spa_result[node] == true_spa_result[node]):
            print(" (Correct)")
        else:
            print(" (Wrong)")
    print(f"{'-'*len(header)}\n")

We compare the results of DFS-based shortest path algorithm and Dijkstra's shortest path algorithm.

In [8]:
cmpr_two_spa_result(10, 10)
cmpr_two_spa_result(20, 20)

Lattice (n=10, k=10)
---------------------------------------
Node : DFS SPA result : True SPA result
---------------------------------------
0 : 0 : 0 (Correct)
1 : 1 : 1 (Correct)
2 : 2 : 1 (Wrong)
3 : 3 : 1 (Wrong)
4 : 4 : 1 (Wrong)
5 : 5 : 1 (Wrong)
6 : 6 : 1 (Wrong)
7 : 7 : 1 (Wrong)
8 : 8 : 1 (Wrong)
9 : 9 : 1 (Wrong)
---------------------------------------

Lattice (n=20, k=20)
---------------------------------------
Node : DFS SPA result : True SPA result
---------------------------------------
0 : 0 : 0 (Correct)
1 : 1 : 1 (Correct)
2 : 2 : 1 (Wrong)
3 : 3 : 1 (Wrong)
4 : 4 : 1 (Wrong)
5 : 5 : 1 (Wrong)
6 : 6 : 1 (Wrong)
7 : 7 : 1 (Wrong)
8 : 8 : 1 (Wrong)
9 : 9 : 1 (Wrong)
10 : 10 : 1 (Wrong)
11 : 11 : 1 (Wrong)
12 : 12 : 1 (Wrong)
13 : 13 : 1 (Wrong)
14 : 14 : 1 (Wrong)
15 : 15 : 1 (Wrong)
16 : 16 : 1 (Wrong)
17 : 17 : 1 (Wrong)
18 : 18 : 1 (Wrong)
19 : 19 : 1 (Wrong)
---------------------------------------

